# Comprehensive Tutorial on Using Pathlib In Python For File System Manipulation

## Introduction

One of the most frustrating aspects of Python up until version 3.4 was file system manipulation. Developers often struggled with tangled strings representing paths. Their code broke frequently due to path inconsistencies on different operating systems (Windows vs. Unix-like). That's when the `pathlib` module was introduced to the standard library.

`pathlib` offers a long-awaited object-oriented approach to path manipulation. It offers a powerful and elegant solution to handling file system paths, ensures platform-agnostic behavior, and promotes code clarity and maintainability. 

The module has matured significantly over the years, making it an essential tool for any Pythonista. This comprehensive tutorial will teach you the features and methods of `pathlib` that will probably enough for 99% of your daily needs. Let's get started.

## Python `os` module vs. `pathlib`

Some of our readers might ask "Why learn a new library when we have the Python `os` module?". That's a fair question. 

Let's say we want to find all `png` files inside a given directory and all its sub-directories (a common task in data science). If we were using the `os` module, we would have to write code like this:

In [ ]:
import os

dir_path = "/home/user/documents"

# Find all text files inside a directory
files = [
    os.path.join(dir_path, f)
    for f in os.listdir(dir_path)
    if os.path.isfile(os.path.join(dir_path, f)) and f.endswith(".png")
]

This snippet has many disadvantages:
1. It is long and unreadable for such a simple operation.
2. Requires knowledge of list comprehensions.
3. It involves string operations which are error-prone.

If we were using `pathlib`, then our code would be much simpler:

In [ ]:
from pathlib import Path

# Create a path object
dir_path = Path(dir_path)

# Find all text files inside a directory
files = list(dir_path.glob("*.png"))

If you continue reading the article, you will discover many more benefits of `pathlib` over the `os` module besides simplicity and readability. So, shall we?

## `Path` objects

The entire `pathlib` library revolves around `Path` objects:

In [2]:
from pathlib import Path

These objects represent file system paths in a structured and - this is key - platform-independent way. Unlike working with raw strings, these objects offer a more user-friendly approach to manipulating file system paths.

We can create `Path` objects in several ways:

1. __From strings__

You can directly create a `Path` object by passing a string that represents a file system path:

In [4]:
file_path_str = "data/union_data.csv"
data_path = Path(file_path_str)

print(type(data_path))

<class 'pathlib.PosixPath'>


2. __From other `path` objects__

Existing `Path` objects can be building blocks to create new paths. You can combine them using operators:

In [10]:
base_path = Path("/home/user")
data_dir = Path("data")

file_path = base_path / data_dir / "prices.csv"  # Combining multiple paths
print(file_path)

/home/user/data/prices.csv


By using a forward slash, you can extend `Path` objects with another object or a string path. 

3. __From the current working directory__

The `Path.cwd()` method gives a fast access to the current working directory as a Path object:

In [9]:
cwd = Path.cwd()

print(cwd)

/home/bexgboost/articles/2024/4_april/8_pathlib


4. __From the home directory__

In [11]:
home = Path.home()

home / "downloads" / "projects"

PosixPath('/home/bexgboost/downloads/projects')

__An important note__: The `Path` class itself doesn't perform any file system operations such as path validation, creating directories or files. It is designed for representing and manipulating paths. To actually interact with the file system (checking existence, reading/writing files), we will have to use special methods of `Path` objects and for some advanced cases, get help from the `os` module. More on this later.

## `Path` components 

Just like a physical address has different parts (street number, city, country, zip code, etc.), a file system path can be broken down into smaller components. `pathlib` allows us to access and manipulate these components using path attributes through dot-notation. 

Here are some common path attributes and how to retrieve them in `pathlib`:

- __Root__: This refers to the top level of the file system (e.g., "/" on Unix-like systems, drive letter like "C:" on Windows).

In [13]:
image_file = home / "downloads" / "midjourney.png"

image_file.root

'/'

- __Parent__: This attribute returns a `Path` object representing the directory containing the current path.

In [14]:
image_file.parent

PosixPath('/home/bexgboost/downloads')

- __name__: This attribute returns the entire filename (including extension) as a string.

In [15]:
image_file.name

'midjourney.png'

- __suffix__: This attribute returns the file extension (including the dot) as a string, or an empty string if there's no extension.

In [16]:
image_file.suffix

'.png'

- __stem__: This attribute returns the file name without the extension. It is useful when converting files to different formats.

In [17]:
image_file.stem

'midjourney'

If you want to split a `Path` object into its components, you can use the `.parts` attribute:

In [18]:
image_file.parts

('/', 'home', 'bexgboost', 'downloads', 'midjourney.png')

If you want these components to be `Path` objects in themselves, you can use the `.parents` attribute, which returns a generator:

In [19]:
list(image_file.parents)

[PosixPath('/home/bexgboost/downloads'),
 PosixPath('/home/bexgboost'),
 PosixPath('/home'),
 PosixPath('/')]

## Common path operations using `pathlib`

`Path` objects have many methods that allows you to efficiently interact with directories and contents. Below, we will look at how to perform some of the most common operations you will face daily.

### Listing directories

The `iterdir()` method allows you to iterate over all entries (files and subdirectories) within a folder given as a `Path` object. It is particularly useful for processing all files in a directory or performing operations on each entry:

In [21]:
cwd = Path.cwd()

for entry in cwd.iterdir():
    # Process the entry here
    ...
    # print(entry)

Since `iterdir()` returns an interator, entries are retrieved on-demand as you go through the loop. 

While iterating through a directory, you may want to focus processing only on files or directories. `Path` objects have methods for checking entry type:

- `is_dir()`: This method returns `True` if the path points to a directory, `False` otherwise.

In [23]:
for entry in cwd.iterdir():
    if entry.is_dir():
        print(entry.name)

.ipynb_checkpoints
data
images


- `is_file()`: This method returns `True` if the path points to a regular file, `False` otherwise.

In [25]:
for entry in cwd.iterdir():
    if entry.is_file():
        print(entry.suffix)

.ipynb


Since `Path` objects only represent paths, sometimes you need to check if a path actually exists using the `.exists()` method:

In [26]:
image_file.exists()

False

### Creating and deleting paths

`pathlib` also offers functionalities for creating and deleting files and directories. Let's see how.

- `mkdir()`: This method creates a new directory at the specified path. By default, it creates the directory in the current working directory.

In [27]:
from pathlib import Path

data_dir = Path("new_data_dir")

# Create the directory 'new_data_dir' in the current working directory
data_dir.mkdir()

- `mkdir(parents=True)`: This method is particularly useful when you want to create a directory structure where some parent directories might not exist. Setting `parents=True` ensures that all necessary parent directories are created along the way.

In [28]:
sub_dir = Path("data/nested/subdirectory")

# Create 'data/nested/subdirectory', even if 'data' or 'nested' don't exist
sub_dir.mkdir(parents=True)

Keep in mind that `mkdir` raises an exception if a directory with the same name already exists:

```python
Path('data').mkdir()
```

```
FileExistsError: [Errno 17] File exists: 'data'
```

- `unlink()`: This method permanently deletes a file represented by the `Path` object. It is recommended to check if a file exists before running this method because it may raise an error:

```python
to_delete = Path("data/prices.csv")

if to_delete.exists():
    to_delete.unlink()
    print(f"Successfully deleted {to_delete.name}")
```

```
Successfully deleted prices.csv
```

- `rmdir()`: This method removes an empty (only empty) directory. If you want to delete a non-empty directory, the easiest way is to use [`shutil` library](https://stackoverflow.com/questions/303200/how-do-i-remove-delete-a-folder-that-is-not-empty) or the terminal.

In [33]:
empty_dir = Path("new_data_dir")

empty_dir.rmdir()

Please, be cautious when using `unlink()` or `rmdir()` as their results are permanent.

## Advanced path manipulation

### Relative vs. absolute paths

### Joining and splitting paths

### Globbing

## Working with files

### Reading files

### Writing files

### File renaming and moving

## Additional functionalities

### Iterating over file trees

### Temporary files and directories

### Permissions and file system information

## Conclusion